In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 425.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=61fa924a99921de79e3127a566139a6b76f93cd595859a479808661ed4665e74
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [3]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.3 MB/s eta 0:00:00


In [4]:
!pip install --upgrade --quiet langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 1.5 MB/s eta 0:00:00


In [5]:
NEO4J_URI="<your-neo4j-uri>"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="<your-neo4j-password>"

In [6]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [7]:
groq_api_key="<your-api-key>"
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x782913036e00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x782913036140>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [8]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [9]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:4])

In [10]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [11]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [12]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [142]:
cypher_generation_template = """
You are an expert Neo4j Cypher translator who converts English to Cypher based on the Neo4j Schema provided, following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE, HAVING keywords in the cypher. Use alias when using the WITH keyword
3. Use only Nodes and relationships mentioned in the schema
4. Never use relationships that are not mentioned in the given schema
5. When asked about projects, Match the properties using case-insensitive matching and the OR-operator, E.g, to find a logistics platform -project, use `toLower(project.summary) contains 'logistics platform' OR toLower(project.name) contains 'logistics platform'`.

schema: {schema}

Examples:
Human: Who are the advisors of Elizabeth I?
Assistant: ```MATCH (elizabeth:Person {id: 'Elizabeth I'})-[:ADVISOR]->(advisor:Person)
RETURN advisor.id, advisor.type```

Human: Who are the siblings of Elizabeth I?
Assistant: ```MATCH (elizabeth:Person {id: 'Elizabeth I'})-[:SIBLING]->(sibling:Person)
RETURN sibling.id, sibling.type```

Human: Who are the parents of Elizabeth I?
Assistant: ```MATCH (elizabeth:Person {id: 'Elizabeth I'})-[:CHILD]->(parent:Person)
RETURN parent.id, parent.type```

Human: Who is the predecessor of Elizabeth I?
Assistant: ```MATCH (elizabeth:Person {id: 'Elizabeth I'})-[:PREDECESSOR]->(predecessor:Person)
RETURN predecessor.id, predecessor.type```

Human: Who are the children of Henry VIII?
Assistant: ```MATCH (child:Person)-[:CHILD]->(henry:Person {id: 'Henry Viii'})
RETURN child.id, child.type```

Human: Who is related to Elizabeth I?
Assistant: ```MATCH (elizabeth:Person {id: 'Elizabeth I'})-[:RELATIVE]->(relative:Person)
RETURN relative.id, relative.type```

Question: {question}
"""

cypher_prompt = PromptTemplate(
    template = cypher_generation_template,
    input_variables = ["schema", "question"]
)

In [86]:
!pip install langchain_google_genai

In [87]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm2 = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key="<your-api-key>",temperature=0)

In [116]:
llm3=ChatGroq(groq_api_key=groq_api_key,model_name="llama3-70b-8192")
llm3

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x78290c5e9900>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x78290c5e8ca0>, model_name='llama3-70b-8192', groq_api_key=SecretStr('**********'))

In [135]:
# mixtral-8x7b-32768
llm4=ChatGroq(groq_api_key=groq_api_key,model_name="mixtral-8x7b-32768")
llm4

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x78290c5b7a90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x78290c5d46a0>, groq_api_key=SecretStr('**********'))

In [166]:
from langchain.prompts.prompt import PromptTemplate
CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
If the provided information is empty, say that you don't know the answer.
Only answer based on the information provided and not other sources.
Final answer should be easily readable and structured.

Question: {question}

Information:
{context}

Question: {question}
Helpful Answer:"""

qa_prompt = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

from neo4j import GraphDatabase
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
        llm=llm4,
        graph=graph,
        verbose=True,
        return_intermediate_steps=True,
        qa_prompt=qa_prompt,
        # cypher_prompt=cypher_prompt

        )

In [147]:
response=chain.invoke({"query":"Who are the siblings of Elizabeth I?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Person)-[:SIBLING]->(s) WHERE e.name = 'Elizabeth I' RETURN s.name
Full Context:
[]

> Finished chain.


{'query': 'Who are the siblings of Elizabeth I?',
 'result': "Elizabeth I had two siblings: Mary I and Edward VI. Mary was her older sister, who also reigned as Queen of England before Elizabeth. Edward was their younger brother, who became King of England after their father's death. I don't have any information about other siblings.\n\n(Note: The information provided was empty, so I rely on generally known facts about Elizabeth I and her family.)",
 'intermediate_steps': [{'query': "MATCH (e:Person)-[:SIBLING]->(s) WHERE e.name = 'Elizabeth I' RETURN s.name"},
  {'context': []}]}

In [167]:
def question_ans(question):
    response=chain.invoke({"query":question})
    print(response)
    # semi_cypher=response.get("intermediate_steps")[0]["query"]
    # print("Semi Cypher:")
    # print(semi_cypher)
    # full_cypher = convert_semi_to_full_cypher(semi_cypher)
    # print("Full Cypher:")
    # print(full_cypher)
    # result=chain.invoke({"query":full_cypher})
    # # return result
    # print(result)



In [150]:
question_ans("Who are the advisors of Elizabeth I?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {id: 'Elizabeth I'})-[:ADVISOR]->(advisor)
RETURN advisor.id, advisor.name
Full Context:
[{'advisor.id': 'William Cecil', 'advisor.name': None}, {'advisor.id': 'Sir Francis Walsingham', 'advisor.name': None}]

> Finished chain.
{'query': 'Who are the advisors of Elizabeth I?', 'result': 'Elizabeth I had several advisors, but I can currently provide information on two of them. \n\nThe first is William Cecil. He was a trusted advisor and a prominent figure during her reign.\n\nThe second is Sir Francis Walsingham. He served as the Principal Secretary and was known for his role in the detection of the Babington Plot, which aimed to assassinate Elizabeth I.\n\nI hope this information is helpful. If you have any more questions, please feel free to ask!', 'intermediate_steps': [{'query': "MATCH (:Person {id: 'Elizabeth I'})-[:ADVISOR]->(advisor)\nRETURN advisor.id, advisor.name"}, {'context': [{'advisor.id': 'Willi

In [155]:
question_ans("Who are the advisors of Elizabeth I?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {id: 'Elizabeth I'})-[:ADVISOR]->(advisor)
RETURN advisor.id, advisor.name
Full Context:
[{'advisor.id': 'William Cecil', 'advisor.name': None}, {'advisor.id': 'Sir Francis Walsingham', 'advisor.name': None}]

> Finished chain.
{'query': 'Who are the advisors of Elizabeth I?', 'result': "Elizabeth I had several advisors during her reign, but I can provide you with information on two of them.\n\nThe first one is William Cecil. He was a prominent figure in Elizabeth I's government, holding positions such as Secretary of State, Lord High Treasurer, and Lord Burghley. He played a crucial role in the Queen's administration and was one of her most trusted advisors.\n\nThe second advisor is Sir Francis Walsingham. He served as Principal Secretary and as the spymaster of Elizabeth I. Sir Francis Walsingham was responsible for uncovering various plots against the Queen and was known for his efforts in ensuring the sec

In [158]:
question_ans("Who are the descendants of Anne Boleyn?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person {id: 'Anne Boleyn'})-[:DAUGHTER|SON*..]->(descendant)
RETURN descendant.id
Full Context:
[]

> Finished chain.
{'query': 'Who are the descendants of Anne Boleyn?', 'result': "I don't have the information to provide a detailed answer on the descendants of Anne Boleyn. However, I can tell you that Anne Boleyn was the second wife of King Henry VIII of England and the mother of Queen Elizabeth I. Unfortunately, she was beheaded in 1536, so she did not have any further descendants.", 'intermediate_steps': [{'query': "MATCH (a:Person {id: 'Anne Boleyn'})-[:DAUGHTER|SON*..]->(descendant)\nRETURN descendant.id"}, {'context': []}]}


In [159]:
question_ans("Who is related to Mary, Queen of Scots?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (mqos:Person)-[r]->(p) WHERE mqos.name = 'Mary, Queen of Scots' RETURN p, type(r)
Full Context:
[]

> Finished chain.
{'query': 'Who is related to Mary, Queen of Scots?', 'result': "Based on the information provided, I don't have any details about who is related to Mary, Queen of Scots. However, Mary, Queen of Scots, is known to be the cousin of Queen Elizabeth I of England, as they both were descendants of King Henry VII of England.", 'intermediate_steps': [{'query': "MATCH (mqos:Person)-[r]->(p) WHERE mqos.name = 'Mary, Queen of Scots' RETURN p, type(r)"}, {'context': []}]}


In [160]:
question_ans("Who are the siblings of Edward VI?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (edward:Person)-[:SIBLING]->(sibling)
WHERE edward.id = 'Edward_VI'
RETURN sibling
Full Context:
[]

> Finished chain.
{'query': 'Who are the siblings of Edward VI?', 'result': 'Edward VI had two sisters, both of them became queens of England. Mary I, also known as Bloody Mary, was the eldest, and Elizabeth I, the Gloriana or the Virgin Queen, was the youngest. There were no brothers.', 'intermediate_steps': [{'query': "MATCH (edward:Person)-[:SIBLING]->(sibling)\nWHERE edward.id = 'Edward_VI'\nRETURN sibling"}, {'context': []}]}


In [161]:
question_ans("Who is the successor of Lady Jane Grey?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Person)-[:SUCCESSOR]->(p2:Person)
WHERE p1.name = 'Lady Jane Grey'
RETURN p2.name
Full Context:
[]

> Finished chain.
{'query': 'Who is the successor of Lady Jane Grey?', 'result': "I don't have enough information to provide an accurate answer to your question. The successor of Lady Jane Grey is a topic that requires specific historical knowledge. In this case, it's important to note that Lady Jane Grey was Queen of England for a very brief period in 1553 before being succeeded by Queen Mary I. If you have any additional context or details, I'd be happy to try and help further.", 'intermediate_steps': [{'query': "MATCH (p1:Person)-[:SUCCESSOR]->(p2:Person)\nWHERE p1.name = 'Lady Jane Grey'\nRETURN p2.name"}, {'context': []}]}


In [164]:
question_ans("Find all children of Henry VIII")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:CHILD]->(c:Person) WHERE p.id = 'Henry VIII' RETURN c
Full Context:
[]

> Finished chain.
{'query': 'Find all children of Henry VIII', 'result': "Henry VIII had six children, but not all of them survived to adulthood. His children were as follows:\n\n1. Princess Mary, also known as Mary I, born on February 18, 1516. She became the Queen of England after her father's death.\n2. Henry FitzRoy, Duke of Richmond and Somerset, born on June 15, 1519. He was an illegitimate son of Henry VIII, recognized by the King, but died at a young age of 17.\n3. Princess Elizabeth, also known as Elizabeth I, born on September 7, 1533. She became the Queen of England after her half-sister Mary I's death.\n4. Edward, Prince of Wales, later King Edward VI, born on October 12, 1537. He reigned as the King of England after his father's death but passed away in his teenage years.\n5. A stillborn son, born on October 29, 1538.\n6.

In [168]:
question_ans("Find all spouses of Henry VIII")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:SPOUSE]->(spouse) WHERE p.id = 'Henry VIII' RETURN spouse
Full Context:
[]

> Finished chain.
{'query': 'Find all spouses of Henry VIII', 'result': "I don't have enough information to provide an accurate answer to your question. However, I can tell you that Henry VIII, the King of England from 1509 to 1547, had six spouses: Catherine of Aragon, Anne Boleyn, Jane Seymour, Anne of Cleves, Catherine Howard, and Catherine Parr.", 'intermediate_steps': [{'query': "MATCH (p:Person)-[:SPOUSE]->(spouse) WHERE p.id = 'Henry VIII' RETURN spouse"}, {'context': []}]}


In [170]:
question_ans("Find the successor of Edward VI")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (person:Person)-[:SUCCESSOR]->(successor:Person)
WHERE person.id = 'Edward VI'
RETURN successor.id
Full Context:
[]

> Finished chain.
{'query': 'Find the successor of Edward VI', 'result': 'I am unable to provide an answer to your question, as I do not have the necessary information. The successor of Edward VI was Lady Jane Grey, but her reign was very short-lived, and she was soon succeeded by Mary I. It is possible that you are asking about a different context or time period. Could you please provide more information to help clarify your question?', 'intermediate_steps': [{'query': "MATCH (person:Person)-[:SUCCESSOR]->(successor:Person)\nWHERE person.id = 'Edward VI'\nRETURN successor.id"}, {'context': []}]}


In [172]:
question_ans("Who are the advisors of Elizabeth I?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {id: 'Elizabeth I'})-[:ADVISOR]->(advisor)
RETURN advisor.id, advisor.name
Full Context:
[{'advisor.id': 'William Cecil', 'advisor.name': None}, {'advisor.id': 'Sir Francis Walsingham', 'advisor.name': None}]

> Finished chain.
{'query': 'Who are the advisors of Elizabeth I?', 'result': "Elizabeth I had several advisors, but I can tell you about two of them. \n\nThe first one is William Cecil. He was a trusted advisor and served as the Secretary of State and Lord High Treasurer. \n\nThe second advisor is Sir Francis Walsingham. He was a spymaster and principal secretary to the Queen. \n\nThese two individuals played significant roles in Elizabeth I's reign.", 'intermediate_steps': [{'query': "MATCH (:Person {id: 'Elizabeth I'})-[:ADVISOR]->(advisor)\nRETURN advisor.id, advisor.name"}, {'context': [{'advisor.id': 'William Cecil', 'advisor.name': None}, {'advisor.id': 'Sir Francis Walsingham', 'advisor.name': N

In [173]:
question_ans("Find all people who were predecessors to someone")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Person)-[:PREDECESSOR]->(p2:Person)
RETURN p1
Full Context:
[{'p1': {'id': 'Mary I'}}]

> Finished chain.
{'query': 'Find all people who were predecessors to someone', 'result': 'Based on the information provided, the only predecessor to someone I can identify is Mary I. Since there is no information about any other individuals, I cannot provide information on any other predecessors.', 'intermediate_steps': [{'query': 'MATCH (p1:Person)-[:PREDECESSOR]->(p2:Person)\nRETURN p1'}, {'context': [{'p1': {'id': 'Mary I'}}]}]}
